## Descriptions:
- Query other tables on shc to see if we can get any worthy information not included in prediction
- Some bigger queries cannot be done in this notebook. use separate SQL file in python instead
- Check the characteristics of the `diff4` cohort (predicted probability difference >= 0.4), compared to the full cohort

**Inputs**:  
- `1_4_cohort_diff_...`
- `1_4_cohort_all_current_dx`
    
**Outputs**: (did not save anything here)

### Importing R libraries

In [1]:
library(bigrquery)  # to query STARR-OMOP (stored in BigQuery) using SQL
library(tidyverse)
library(lubridate)

# options(repr.matrix.max.rows=250, repr.matrix.max.cols=30)

Warning message:
"package 'bigrquery' was built under R version 4.0.5"
-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.0.4     v dplyr   1.0.2
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.0

Warning message:
"package 'ggplot2' was built under R version 4.0.5"
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union




### Set up and run queries

In [17]:
# CREDENTIALS depending on LOCATIONS:
# credential <- paste0("/home/", "minh084", "/.config/gcloud/application_default_credentials.json")

# local computer
# credential <- "C:/Users/User/AppData/Roaming/gcloud/application_default_credentials.json"

# Nero onprem
# credential <- "/home/minh084/.config/gcloud/application_default_credentials.json"

# Nero gcp notebook
credential <- "/home/jupyter/.config/gcloud/application_default_credentials.json"

project_id <- "som-nero-phi-jonc101"

Sys.setenv(GOOGLE_APPLICATION_CREDENTIALS = credential)
Sys.setenv(GCLOUD_PROJECT = project_id)
gargle::credentials_app_default()

NULL

In [18]:
library(DBI)
con <- dbConnect(
  bigrquery::bigquery(),
  project = project_id,
  dataset = "shc_core" #, billing = project_id
)
con 
dbListTables(con)

<BigQueryConnection>
  Dataset: som-nero-phi-jonc101.shc_core
  Billing: som-nero-phi-jonc101

[1] "adt"                 "allergy"             "culture_sensitivity"
 [4] "demographic"         "dep_map"             "diagnosis_code"     
 [7] "drg_code"            "encounter"           "family_hx"          
[10] "flowsheet"           "lab_result"          "lda"                
[13] "mapped_meds"         "mar"                 "med_orderset"       
[16] "ndc_code"            "order_med"           "order_proc"         
[19] "order_quest"         "proc_note"           "proc_orderset"      
[22] "procedure_code"      "prov_map"            "social_hx"          
[25] "treatment_team"

### ADT, Demographic, and Diagnosis tables

In [19]:
q = "
SELECT c.anon_id, c.pat_enc_csn_id_coded, c.admit_time, c.first_label, c.death_24hr_recent_label,
        c.death_24hr_max_label, c.pred_first, c.pred_death_24hr_recent,
    adt.event_type_c, adt.event_type, adt.pat_class_c, adt.pat_class, adt.base_pat_class_c, adt.base_pat_class,
    adt.pat_service_c, adt.pat_service, adt.pat_lvl_of_care_c, adt.pat_lv_of_care, adt.accommodation_c, adt.accomodation, 
    adt.in_event_type_c, adt.in_event_type, adt.out_event_type_c, adt.out_event_type, adt.from_base_class_c, adt.from_base_class,
    adt.to_base_class_c, adt.to_base_class, adt.seq_num_in_enc, adt.seq_num_in_bed_min
FROM 
    `som-nero-phi-jonc101.triageTD.1_4_cohort_diff_full_features` as c
JOIN 
    `som-nero-phi-jonc101.shc_core.adt` as adt
ON 
    (c.anon_id = adt.anon_id and c.pat_enc_csn_id_coded = adt.pat_enc_csn_id_coded)
"
# submit the query:
update.packages('Rcpp')
df <- dbGetQuery(con, q)
dim(df)
colnames(df)

Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”


[1] 1437248      30

[1] "anon_id"                 "pat_enc_csn_id_coded"   
 [3] "admit_time"              "first_label"            
 [5] "death_24hr_recent_label" "death_24hr_max_label"   
 [7] "pred_first"              "pred_death_24hr_recent" 
 [9] "event_type_c"            "event_type"             
[11] "pat_class_c"             "pat_class"              
[13] "base_pat_class_c"        "base_pat_class"         
[15] "pat_service_c"           "pat_service"            
[17] "pat_lvl_of_care_c"       "pat_lv_of_care"         
[19] "accommodation_c"         "accomodation"           
[21] "in_event_type_c"         "in_event_type"          
[23] "out_event_type_c"        "out_event_type"         
[25] "from_base_class_c"       "from_base_class"        
[27] "to_base_class_c"         "to_base_class"          
[29] "seq_num_in_enc"          "seq_num_in_bed_min"

In [21]:
q = "
SELECT c.anon_id, c.pat_enc_csn_id_coded, 
    x.INTRPTR_NEEDED_YN, x.CHARLSON_SCORE, x.N_HOSPITALIZATIONS, x.DAYS_IN_HOSPITAL
FROM 
    `som-nero-phi-jonc101.triageTD.1_4_cohort_diff_full_features` as c
JOIN 
    `som-nero-phi-jonc101.shc_core.demographic` as x
ON 
    (c.anon_id = x.anon_id)
"
# submit the query:
update.packages('Rcpp')
x1 <- dbGetQuery(con, q)
dim(x1)
colnames(x1)

Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”


[1] 71107     6

[1] "anon_id"              "pat_enc_csn_id_coded" "INTRPTR_NEEDED_YN"   
[4] "CHARLSON_SCORE"       "N_HOSPITALIZATIONS"   "DAYS_IN_HOSPITAL"

In [11]:
# this doesn't seem right, redo with a separate SQL file as it is too large to be in R notebook
q = "
SELECT distinct c.anon_id, c.pat_enc_csn_id_coded, 
    x.line, x.dx_name, x.primary, x.chronic, x.principal, x.hospital_pl, x.ed, x.present_on_adm
FROM 
    `som-nero-phi-jonc101.triageTD.1_4_cohort_24hrpreadmit_diff_full_features` as c
JOIN 
    `som-nero-phi-jonc101.shc_core.diagnosis_code` as x
ON 
    (c.anon_id = x.anon_id and c.pat_enc_csn_id_coded = x.pat_enc_csn_id_jittered)
"
# submit the query:
update.packages('Rcpp')
x2 <- dbGetQuery(con, q)
dim(x2)
colnames(x2)

Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”


[1] 8922   10

[1] "anon_id"              "pat_enc_csn_id_coded" "line"                
 [4] "dx_name"              "primary"              "chronic"             
 [7] "principal"            "hospital_pl"          "ed"                  
[10] "present_on_adm"

In [12]:
head(x2)

anon_id,pat_enc_csn_id_coded,line,dx_name,primary,chronic,principal,hospital_pl,ed,present_on_adm
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
JCe5f7b6,NA,1,Hyponatremia,Y,N,NA,NA,Y,NA
JCdc571d,NA,6,Cough,N,N,NA,NA,N,NA
JCe35ee6,NA,11,"Dementia without behavioral disturbance, unspecified dementia type (CMS-HCC)",N,Y,NA,NA,N,NA
JCd1f333,NA,3,Diabetic ketoacidosis without coma associated with type 1 diabetes mellitus (CMS-HCC),N,N,NA,NA,N,NA
JCe09f7a,NA,1,"Vomiting, unspecified",NA,NA,NA,NA,NA,NA
JCe439dd,NA,7,Bipolar 1 disorder (CMS-HCC),N,N,NA,NA,N,NA


In [2]:
# directories
datadir = "../../DataTD"
cohortdir = "../../OutputTD/1_cohort"
featuredir = "../../OutputTD/2_features"
modeldir4 = "../../OutputTD/3_models/1_4_cohort"
# modeldir4preadmit = "../../OutputTD/3_models/1_4_cohort_24hrpreadmit"
tabledir = "../../OutputTD/4_tables"

options(repr.matrix.max.rows=200, repr.matrix.max.cols=30)

In [3]:
cohort <- read.csv(file.path(modeldir4, "1_4_cohort_diff_full_features.csv")) %>% distinct()

nrow(cohort) # 71589 vs 69334
nrow(cohort %>% distinct())
nrow(cohort %>% select(anon_id) %>% distinct()) # 301
nrow(cohort %>% select(pat_enc_csn_id_coded) %>% distinct()) # 323

# count anon_id and csn, count unique in each group, but possible to have same ids in different groups
cohort %>%  distinct() %>% group_by(abs_diff0_24) %>% 
            summarise(nrows = n(), count_csn = n_distinct(pat_enc_csn_id_coded), count_mrn = n_distinct(anon_id)) %>% 
            arrange(-abs_diff0_24) %>% mutate(cum_csn = cumsum(count_csn), cum_mrn = cumsum(count_mrn))

[1] 71586

[1] 71586

[1] 301

[1] 323

`summarise()` ungrouping output (override with `.groups` argument)



abs_diff0_24,nrows,count_csn,count_mrn,cum_csn,cum_mrn
<dbl>,<int>,<int>,<int>,<int>,<int>
0.7,560,3,3,3,3
0.6,1042,7,7,10,10
0.5,6559,27,24,37,34
0.4,19690,80,77,117,111
0.3,43735,206,204,323,315


In [4]:
head(cohort, 1)
colnames(cohort)

,anon_id,pat_enc_csn_id_coded,admit_time,first_label,death_24hr_recent_label,death_24hr_max_label,pred_first,pred_death_24hr_recent,abs_diff0_24,diff0_True,diff24_True,feature_type,features,values,time,hr_before_admit
,<chr>,<dbl>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,JCcd3b9d,131267453587,2019-04-06 23:15:00,1,1,1,0.5339708,0.2470261,0.3,-0.47,-0.75,Procedures,LIMITED ULTRASOUND- CARDIAC TRANSTHORACIC ECHO,1,NA,NA


[1] "anon_id"                 "pat_enc_csn_id_coded"   
 [3] "admit_time"              "first_label"            
 [5] "death_24hr_recent_label" "death_24hr_max_label"   
 [7] "pred_first"              "pred_death_24hr_recent" 
 [9] "abs_diff0_24"            "diff0_True"             
[11] "diff24_True"             "feature_type"           
[13] "features"                "values"                 
[15] "time"                    "hr_before_admit"

In [9]:
## check csn=131274348480
cohort %>% filter(pat_enc_csn_id_coded == 131274348480, feature_type=="demo")

anon_id,pat_enc_csn_id_coded,admit_time,first_label,death_24hr_recent_label,death_24hr_max_label,pred_first,pred_death_24hr_recent,abs_diff0_24,diff0_True,diff24_True,feature_type,features,values,time,hr_before_admit
<chr>,<dbl>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>
JCda4f94,131274348480,2019-08-07 20:38:00,0,0,0,0.8106468,0.4368035,0.4,0.81,0.44,demo,ESI_i,3,NA,NA
JCda4f94,131274348480,2019-08-07 20:38:00,0,0,0,0.8106468,0.4368035,0.4,0.81,0.44,demo,delta_ESI,0,NA,NA
JCda4f94,131274348480,2019-08-07 20:38:00,0,0,0,0.8106468,0.4368035,0.4,0.81,0.44,demo,gender,1,NA,NA
JCda4f94,131274348480,2019-08-07 20:38:00,0,0,0,0.8106468,0.4368035,0.4,0.81,0.44,demo,age,89,NA,NA
JCda4f94,131274348480,2019-08-07 20:38:00,0,0,0,0.8106468,0.4368035,0.4,0.81,0.44,demo,insurance,1,NA,NA
JCda4f94,131274348480,2019-08-07 20:38:00,0,0,0,0.8106468,0.4368035,0.4,0.81,0.44,demo,English,1,NA,NA
JCda4f94,131274348480,2019-08-07 20:38:00,0,0,0,0.8106468,0.4368035,0.4,0.81,0.44,demo,Height_i,173,NA,NA
JCda4f94,131274348480,2019-08-07 20:38:00,0,0,0,0.8106468,0.4368035,0.4,0.81,0.44,demo,delta_H,0,NA,NA
JCda4f94,131274348480,2019-08-07 20:38:00,0,0,0,0.8106468,0.4368035,0.4,0.81,0.44,demo,Weight_i,62,NA,NA


In [13]:
# check different visits
cohort0 <- read.csv(file.path(cohortdir, "1_4_cohort.csv")) %>% distinct()

# there's on in 2018 but was "Place in Observation" under Other orders/ Description
cohort0 %>% filter(anon_id == 'JCda4f94') %>% distinct(pat_enc_csn_id_coded, .keep_all = TRUE)

anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label_max24,label_24hr_recent,admit_label,has_admit_label,died_within_24hrs,death_24hr_max_label,death_24hr_recent_label,first_label,first_label_minutes_since_admit,acute_to_critical_label_recent,critical_to_acute_label_recent,acute_to_critical_label_max,critical_to_acute_label_max
<chr>,<dbl>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<int>,<int>
JCda4f94,131274348480,40433348,2019-08-07 20:38:00+00:00,0,0,0,1,0,0,0,0,0,0,0,0,0


### Checking diagnosis to compare with total cohort

In [5]:
# all dx from 1_4_cohort, SQL query
dx <- read.csv(file.path(datadir, "1_4_cohort_all_current_dx.csv"))
nrow(dx) #1239918

ndx = length(unique(dx$pat_enc_csn_id_coded)) # full cohort is 43980
ndx

[1] 1239918

[1] 43932

In [32]:
# 47246
dx %>% distinct(pat_enc_csn_id_coded, dx_name) %>% count(dx_name, sort=TRUE) %>% 
    mutate(perc = round(100*n/47246, 2)) %>% head(20)

,dx_name,n,perc
,<chr>,<int>,<dbl>
1,Other long term (current) drug therapy,12409,26.26
2,"Hyperlipidemia, unspecified",11532,24.41
3,Essential (primary) hypertension,10691,22.63
4,Personal history of nicotine dependence,9969,21.10
5,Long term (current) use of aspirin,7188,15.21
6,Gastro-esophageal reflux disease without esophagitis,6725,14.23
7,"Acute kidney failure, unspecified",6538,13.84
8,"Major depressive disorder, single episode, unspecified",5719,12.10
9,Long term (current) use of anticoagulants,5456,11.55


In [34]:
# dx present in the ED and identified as the primary problems, 4632
dx %>% filter(ed=="Y", primary=="Y") %>% distinct(pat_enc_csn_id_coded, dx_name) %>% 
        count(dx_name, sort=TRUE) %>% 
        mutate(perc = round(100*n/4632, 2)) %>% head(20)

,dx_name,n,perc
,<chr>,<int>,<dbl>
1,Suicidal ideation,407,8.79
2,Pneumonia due to organism,302,6.52
3,"Sepsis, due to unspecified organism",277,5.98
4,Hyponatremia,274,5.92
5,Non-ST elevation myocardial infarction (NSTEMI) (CMS-HCC),227,4.90
6,SBO (small bowel obstruction) (CMS-HCC),222,4.79
7,Acute chest pain,211,4.56
8,Small bowel obstruction (CMS-HCC),205,4.43
9,Neutropenic fever (CMS-HCC),194,4.19


In [8]:
diff4 <- cohort %>% filter(abs_diff0_24 >=0.4) 

In [9]:
diff4 %>% group_by(abs_diff0_24) %>% 
            summarise(nrows = n(), count_csn = n_distinct(pat_enc_csn_id_coded), count_mrn = n_distinct(anon_id)) %>% 
            arrange(-abs_diff0_24) %>% mutate(cum_csn = cumsum(count_csn), cum_mrn = cumsum(count_mrn))

abs_diff0_24,nrows,count_csn,count_mrn,cum_csn,cum_mrn
<dbl>,<int>,<int>,<int>,<int>,<int>
0.7,560,3,3,3,3
0.6,1042,7,7,10,10
0.5,6559,27,24,37,34
0.4,19690,80,77,117,111


In [36]:
# cohort transfers with diagnoses
dx_diff <- left_join(diff4, dx)
nrow(dx_diff)
ndf = length(unique(dx_diff$pat_enc_csn_id_coded))
ndf
length(unique(dx_diff$anon_id))

# 1222
dx_diff %>% distinct(pat_enc_csn_id_coded, dx_name) %>% count(dx_name, sort=TRUE) %>% 
    mutate(perc = round(100*n/1222, 2)) %>% head(20)

Joining, by = c("anon_id", "pat_enc_csn_id_coded")



[1] 849920

[1] 117

[1] 105

,dx_name,n,perc
,<chr>,<int>,<dbl>
1,Other long term (current) drug therapy,50,4.09
2,"Hyperlipidemia, unspecified",49,4.01
3,Long term (current) use of insulin,42,3.44
4,Essential (primary) hypertension,40,3.27
5,Long term (current) use of aspirin,32,2.62
6,"Major depressive disorder, single episode, unspecified",27,2.21
7,Atherosclerotic heart disease of native coronary artery without angina pectoris,26,2.13
8,Dehydration,25,2.05
9,Personal history of nicotine dependence,25,2.05


In [12]:
# only 117 of obs above 0.4
# all visits in diff4 have diagnoses

ndfn = nrow(dx_diff %>% distinct(pat_enc_csn_id_coded, dx_name))
ndfn

length(setdiff(dx_diff$pat_enc_csn_id_coded, dx$pat_enc_csn_id_coded))
length(setdiff(dx$pat_enc_csn_id_coded, dx_diff$pat_enc_csn_id_coded))

colnames(dx_diff)

[1] 2926

[1] 0

[1] 43815

[1] "anon_id"                 "pat_enc_csn_id_coded"   
 [3] "admit_time"              "first_label"            
 [5] "death_24hr_recent_label" "death_24hr_max_label"   
 [7] "pred_first"              "pred_death_24hr_recent" 
 [9] "transfer"                "abs_diff0_24"           
[11] "diff0_True"              "diff24_True"            
[13] "feature_type"            "features"               
[15] "values"                  "time"                   
[17] "hr_before_admit"         "line"                   
[19] "dx_name"                 "primary"                
[21] "chronic"                 "principal"              
[23] "hospital_pl"             "ed"                     
[25] "present_on_adm"

In [13]:
summary(dx_diff$line)

dx_diff %>% count(primary, sort=TRUE)
dx_diff %>% count(chronic, sort=TRUE)
dx_diff %>% count(hospital_pl,  sort=TRUE)
dx_diff %>% count(principal, sort=TRUE)
dx_diff %>% count(ed, sort=TRUE)
dx_diff %>% count(present_on_adm, sort=TRUE)
# dx_diff %>% count(dx_name, sort=TRUE) %>% head(20)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
   1.00    4.00    9.00   12.36   19.00   53.00    5718 

primary,n
<chr>,<int>
,709610
N,112884
Y,27426


chronic,n
<chr>,<int>
,704997
N,140310
Y,4613


hospital_pl,n
<chr>,<int>
,844202
Y,5493
N,225


principal,n
<chr>,<int>
,844202
N,4675
Y,1043


ed,n
<chr>,<int>
,709610
N,90503
Y,49807


present_on_adm,n
<chr>,<int>
Yes,457212
Exempt from POA reporting,181645
,174002
No,34797
Unknown,2264


In [51]:
# dx_diff %>% filter(ed == "Y", primary=="Y") %>% count(dx_name, sort=TRUE) %>% head(20)

In [40]:
dx_diff %>% filter(ed=="Y", primary=="Y") %>% distinct(pat_enc_csn_id_coded, dx_name) %>% 
            count(dx_name, sort=TRUE) %>% filter(n>1)

dx_name,n
<chr>,<int>
Diabetic ketoacidosis without coma associated with type 1 diabetes mellitus (CMS-HCC),7
Diabetic ketoacidosis without coma associated with other specified diabetes mellitus (CMS-HCC),6
Diabetic ketoacidosis without coma associated with diabetes mellitus due to underlying condition (CMS-HCC),5
Diabetic ketoacidosis without coma associated with type 2 diabetes mellitus (CMS-HCC),5
"ST elevation myocardial infarction (STEMI), unspecified artery (CMS-HCC)",3
Diabetic ketoacidosis with coma associated with type 1 diabetes mellitus (CMS-HCC),2
Hyperglycemic hyperosmolar nonketotic coma (CMS-HCC),2
Hypertensive emergency,2
Intracranial hemorrhage (CMS-HCC),2


In [45]:
# from 1 to 0, 533
dx_diff %>% filter(first_label==1, death_24hr_recent_label==0) %>%
            distinct(pat_enc_csn_id_coded, dx_name) %>% count(dx_name, sort=TRUE) %>% 
            mutate(perc = round(100*n/533, 2)) %>% head(10)

,dx_name,n,perc
,<chr>,<int>,<dbl>
1,"Hyperlipidemia, unspecified",20,3.75
2,Long term (current) use of insulin,20,3.75
3,Essential (primary) hypertension,18,3.38
4,Long term (current) use of aspirin,17,3.19
5,Other long term (current) drug therapy,17,3.19
6,Dehydration,15,2.81
7,Type 1 diabetes mellitus with ketoacidosis without coma,13,2.44
8,Diabetic ketoacidosis without coma associated with type 1 diabetes mellitus (CMS-HCC),12,2.25
9,"Major depressive disorder, single episode, unspecified",11,2.06


In [42]:
# from 1 to 0 
dx_diff %>% filter(first_label==1, death_24hr_recent_label==0, ed=="Y", primary=="Y") %>%
            distinct(pat_enc_csn_id_coded, dx_name) %>% count(dx_name, sort=TRUE) #%>%

dx_name,n
<chr>,<int>
Diabetic ketoacidosis without coma associated with type 1 diabetes mellitus (CMS-HCC),5
Diabetic ketoacidosis without coma associated with diabetes mellitus due to underlying condition (CMS-HCC),4
Diabetic ketoacidosis without coma associated with other specified diabetes mellitus (CMS-HCC),4
Diabetic ketoacidosis without coma associated with type 2 diabetes mellitus (CMS-HCC),3
Intracranial hemorrhage (CMS-HCC),2
Subdural hematoma (CMS-HCC),2
Dehydration,1
Diabetic ketoacidosis with coma associated with type 1 diabetes mellitus (CMS-HCC),1
"Hypertension, unspecified type",1


In [49]:
# from 0 to 1 (non-ICUs to ICUs), 111
dx_diff %>% filter(first_label==0, death_24hr_recent_label==1) %>%
            distinct(pat_enc_csn_id_coded, dx_name) %>% count(dx_name, sort=TRUE) %>% 
            mutate(perc = round(100*n/111, 2)) %>% filter(n>1)

dx_name,n,perc
<chr>,<int>,<dbl>
Acute respiratory failure with hypoxia,3,2.7
Acidosis,2,1.8
Acute on chronic combined systolic (congestive) and diastolic (congestive) heart failure,2,1.8
Anemia in neoplastic disease,2,1.8
Gastro-esophageal reflux disease without esophagitis,2,1.8
Hypo-osmolality and hyponatremia,2,1.8
Long term (current) use of anticoagulants,2,1.8
Other cirrhosis of liver,2,1.8


In [50]:
# from 0 to 1 (non-ICUs to ICUs)
dx_diff %>% filter(first_label==0, death_24hr_recent_label==1, ed=="Y", primary=="Y") %>%
            distinct(pat_enc_csn_id_coded, dx_name) %>% count(dx_name)

dx_name,n
<chr>,<int>
Acute upper respiratory infection,1
Cardiopulmonary arrest with successful resuscitation (CMS-HCC),1
"ST elevation myocardial infarction (STEMI), unspecified artery (CMS-HCC)",1


### ADT

In [51]:
adt <- read.csv(file.path(tabledir, "4_2_diff_adt.csv"))
dim(adt) # 1437248
colnames(adt)

[1] 1437248      30

[1] "anon_id"                 "pat_enc_csn_id_coded"   
 [3] "admit_time"              "first_label"            
 [5] "death_24hr_recent_label" "death_24hr_max_label"   
 [7] "pred_first"              "pred_death_24hr_recent" 
 [9] "event_type_c"            "event_type"             
[11] "pat_class_c"             "pat_class"              
[13] "base_pat_class_c"        "base_pat_class"         
[15] "pat_service_c"           "pat_service"            
[17] "pat_lvl_of_care_c"       "pat_lv_of_care"         
[19] "accommodation_c"         "accomodation"           
[21] "in_event_type_c"         "in_event_type"          
[23] "out_event_type_c"        "out_event_type"         
[25] "from_base_class_c"       "from_base_class"        
[27] "to_base_class_c"         "to_base_class"          
[29] "seq_num_in_enc"          "seq_num_in_bed_min"

In [52]:
cohort %>% distinct(pat_enc_csn_id_coded, first_label, death_24hr_recent_label) %>% count(first_label, death_24hr_recent_label, sort=T)
adt %>% count(event_type, event_type_c, sort=TRUE)
adt %>% count(pat_class, pat_class_c, sort=TRUE)
adt %>% count(base_pat_class, base_pat_class_c, sort=TRUE)
adt %>% count(pat_service, pat_service_c, sort=TRUE)
adt %>% count(pat_lv_of_care, pat_lvl_of_care_c, sort=TRUE)
adt %>% count(accomodation, accommodation_c, sort=TRUE)
adt %>% count(in_event_type, in_event_type_c, sort=TRUE)
adt %>% count(out_event_type, out_event_type_c, sort=TRUE)
adt %>% count(from_base_class, from_base_class_c, sort=TRUE)
adt %>% count(to_base_class, to_base_class_c, sort=TRUE)
adt %>% count(seq_num_in_enc, sort=TRUE)
adt %>% count(seq_num_in_bed_min, sort=TRUE)

first_label,death_24hr_recent_label,n
<int>,<int>,<int>
1,1,113
0,0,102
1,0,90
0,1,18


event_type,event_type_c,n
<chr>,<int>,<int>
Census,6,450404
Transfer In,3,308795
Transfer Out,4,308795
Patient Update,5,228035
Admission,1,71107
Discharge,2,70112


pat_class,pat_class_c,n
<chr>,<int>,<int>
Inpatient,126,1232940
Emergency Services,112,202329
Observation,128,1328
OP Surgery/Procedure,122,651


base_pat_class,base_pat_class_c,n
<chr>,<int>,<int>
NA,NA,1294153
Emergency,3,71107
Inpatient,1,71107
Outpatient,2,881


pat_service,pat_service_c,n
<chr>,<int>,<int>
Critical Care,151,241846
Medicine,39,171188
Emergency,100,151907
Cardiology,155,129561
Emergency Medicine,187,111861
General Medicine (University),153,96646
Trauma,72,76396
General Surgery,59,72896
Cardiac Surgery,58,54242


...

out_event_type,out_event_type_c,n
<chr>,<int>,<int>
Census,6,450404
NA,NA,379902
Transfer Out,4,308634
Patient Update,5,156208
Discharge,2,142100


from_base_class,from_base_class_c,n
<chr>,<int>,<int>
Inpatient,1,1162497
Emergency,3,202329
NA,0,71107
NA,4,664
Outpatient,2,651


to_base_class,to_base_class_c,n
<chr>,<int>,<int>
Inpatient,1,1163436
Emergency,3,202168
NA,0,70112
Outpatient,2,868
NA,4,664


seq_num_in_enc,n
<int>,<int>
1,71107
2,71107
3,71107
4,71107
5,71107
6,70390
7,69868
8,69512
9,68276


seq_num_in_bed_min,n
<int>,<int>
1,1421316
2,15538
3,394
